# Анализ данных для агентства недвижимости

### Введение

Агентству нужен анализ объявлений о продаже жилой недвижимости в Санкт-Петербурге и Ленинградской области, чтобы найти самые перспективные сегменты недвижимости.

Вам предстоит работать с данными схемы real_estate. В ней находится архивная информация объявлений о продаже жилой недвижимости в Санкт-Петербурге и Ленинградской области за несколько лет.

### Описание данных

Таблица `advertisement` cодержит информацию об объявлениях:
- `id` — идентификатор объявления (первичный ключ).
- `first_day_exposition` — дата подачи объявления.
- `days_exposition` — длительность нахождения объявления на сайте (в днях).
- `last_price` — стоимость квартиры в объявлении, в руб.

Таблица `flats` cодержит информацию о квартирах:
- `id` — идентификатор квартиры (первичный ключ, связан с первичным ключом id таблицы advertisement).
- `city_id` — идентификатор города (внешний ключ, связан с city_id таблицы city).
- `type_id` — идентификатор типа населённого пункта (внешний ключ, связан с type_id таблицы type).
- `total_area` — общая площадь квартиры, в кв. метрах.
- `rooms` — число комнат.
- `ceiling_height` — высота потолка, в метрах.
- `floors_total` — этажность дома, в котором находится квартира.
- `living_area` — жилая площадь, в кв. метрах.
- `floor` — этаж квартиры.
- `is_apartment` — указатель, является ли квартира апартаментами (1 — является, 0 — не является).
- `open_plan` — указатель, имеется ли в квартире открытая планировка (1 — открытая планировка квартиры, 0 — открытая планировка отсутствует).
- `kitchen_area` — площадь кухни, в кв. метрах.
- `balcony` — количество балконов в квартире.
- `airports_nearest` — расстояние до ближайшего аэропорта, в метрах.
- `parks_around3000` — число парков в радиусе трёх километров.
- `ponds_around3000` — число водоёмов в радиусе трёх километров.

Таблица `city` cодержит информацию о городах:
- `city_id` — идентификатор населённого пункта (первичный ключ).
- `city` — название населённого пункта.

Таблица `type` cодержит информацию о городах:
- `type_id` — идентификатор типа населённого пункта (первичный ключ).
- `type` — название типа населённого пункта.

---
### Задача 1. Время активности объявлений

Чтобы спланировать эффективную бизнес-стратегию на рынке недвижимости, заказчику нужно определить - по времени активности объявления - самые привлекательные для работы сегменты недвижимости Санкт-Петербурга и городов Ленинградской области.

Результат запроса должен ответить на такие вопросы:
1. Какие сегменты рынка недвижимости Санкт-Петербурга и городов Ленинградской области имеют наиболее короткие или длинные сроки активности объявлений?
2. Какие характеристики недвижимости, включая площадь недвижимости, среднюю стоимость квадратного метра, количество комнат и балконов и другие параметры, влияют на время активности объявлений? Как эти зависимости варьируют между регионами?
3. Есть ли различия между недвижимостью Санкт-Петербурга и Ленинградской области по полученным результатам?

```PostgreSQL
-- Определим аномальные значения (выбросы) по значению перцентилей
WITH limits AS 
    (
    SELECT  
        PERCENTILE_DISC(0.99) WITHIN GROUP (ORDER BY total_area) AS total_area_limit,
        PERCENTILE_DISC(0.99) WITHIN GROUP (ORDER BY rooms) AS rooms_limit,
        PERCENTILE_DISC(0.99) WITHIN GROUP (ORDER BY balcony) AS balcony_limit,
        PERCENTILE_DISC(0.99) WITHIN GROUP (ORDER BY ceiling_height) AS ceiling_height_limit_h,
        PERCENTILE_DISC(0.01) WITHIN GROUP (ORDER BY ceiling_height) AS ceiling_height_limit_l
    FROM real_estate.flats     
    ),   
-- Найдём id объявлений, которые не содержат выбросы
filtered_id AS 
    (
    SELECT id
    FROM real_estate.flats  
    WHERE 
        total_area < (SELECT total_area_limit FROM limits)
        AND (rooms < (SELECT rooms_limit FROM limits) OR rooms IS NULL)
        AND (balcony < (SELECT balcony_limit FROM limits) OR balcony IS NULL)
        AND ((ceiling_height < (SELECT ceiling_height_limit_h FROM limits)
        AND ceiling_height > (SELECT ceiling_height_limit_l FROM limits)) OR ceiling_height IS NULL)
    ),
-- Добавим регион и категорию активности объявления. 
-- Оставим только объявления в городах, уберем объявления с выбросами и которые еще не сняты. 
region_segment AS 
    (
    SELECT 
    	CASE
    		WHEN c.city='Санкт-Петербург' THEN 'Санкт-Петербург'
    		ELSE 'ЛенОбл'
    	END AS region,
    	CASE
    		WHEN a.days_exposition >= 1 AND a.days_exposition <= 30 THEN '1-30 дней'
    		WHEN a.days_exposition <= 90 THEN '31-90 дней'
    		WHEN a.days_exposition <= 180 THEN '91-180 дней'
    		WHEN a.days_exposition >= 181 THEN 'от 181 дней и более'
    	END AS activ_segment,
    	*
    FROM real_estate.flats AS f
    JOIN real_estate.city AS c USING(city_id)
    JOIN real_estate.advertisement AS a USING(id)
    JOIN real_estate.type AS t USING(type_id)
    WHERE 
        f.id IN (SELECT * FROM filtered_id) 
        AND t.type='город' 
        AND a.days_exposition IS NOT NULL
    )
-- Расчитыеваем основные показатели
SELECT
	region,
	activ_segment,
	COUNT(id) AS count_adv,
	ROUND(COUNT(id)::NUMERIC / SUM(COUNT(id)) OVER(PARTITION BY region), 4) AS share_adv_in_region,
	ROUND(AVG(last_price / total_area::NUMERIC)) AS avg_cost_one_m,
	ROUND(AVG(total_area::NUMERIC),1) AS avg_total_area,
	ROUND(AVG(rooms::NUMERIC), 1) AS avg_cnt_rooms,
	ROUND(AVG(balcony::NUMERIC), 2) AS avg_cnt_balcony,
	ROUND(AVG(ceiling_height::NUMERIC), 2) AS avg_ceiling_height,
	ROUND(AVG(parks_around3000::NUMERIC), 2) AS avg_cnt_parks,
	ROUND(AVG(ponds_around3000::NUMERIC), 2) AS avg_cnt_ponds
FROM region_segment
GROUP BY region, activ_segment
ORDER BY region DESC, activ_segment;

```

**Результат:**

region         |activ_segment      |count_adv|share_adv_in_region|avg_cost_one_m|avg_total_area|avg_cnt_rooms|avg_cnt_balcony|avg_ceiling_height|avg_cnt_parks|avg_cnt_ponds|
---------------|-------------------|---------|-------------------|--------------|--------------|-------------|---------------|------------------|-------------|-------------|
Санкт-Петербург|1-30 дней          |     2168|             0.1929|      110569.0|          54.4|          1.9|           1.07|              2.76|         0.56|         0.72|
Санкт-Петербург|31-90 дней         |     3236|             0.2879|      111573.0|          56.7|          1.9|           1.01|              2.77|         0.58|         0.77|
Санкт-Петербург|91-180 дней        |     2254|             0.2006|      111939.0|          60.6|          2.0|           0.95|              2.79|         0.60|         0.75|
Санкт-Петербург|от 181 дней и более|     3581|             0.3186|      115457.0|          66.2|          2.2|           0.92|              2.83|         0.68|         0.87|
ЛенОбл         |1-30 дней          |      397|             0.1438|       73275.0|          48.7|          1.8|           1.07|              2.70|         0.84|         0.91|
ЛенОбл         |31-90 дней         |      917|             0.3322|       67573.0|          50.9|          1.9|           0.96|              2.71|         0.84|         1.17|
ЛенОбл         |91-180 дней        |      556|             0.2014|       69846.0|          51.8|          1.9|           0.92|              2.70|         0.89|         1.04|
ЛенОбл         |от 181 дней и более|      890|             0.3225|       68297.0|          55.4|          2.0|           0.90|              2.72|         0.77|         1.12|



1. В Санкт-Петербурге квартиры с меньшей площадью и количеством комнат имеют более короткий срок объявлений. В Ленинградской области квартиры с меньшей площадью, также имеют более короткий срок объявлений. Большие квартиры продаются дольше в обоих исследуемых субъектах.

2. В Санкт-Петербурге квартиры с большей площадью, большим количеством комнат, более высокой ценой за кв.метр, с более высокими потолками, меньшим количеством балконов и у которых рядом с домом больше парков и водоемов продаются дольше. Дополнительный комфорт выражается в более высокой цене, поэтому квартиры более высокого уровня продаются дольше. На объявления со сроком активности от 6 мес. приходится 31,9% от общего числа всех объявлений по Санкт-Петербургу – это самая большая группа, далее следует группа со сроком 1-3 мес. на которую приходится 28,8%.

    В Ленинградской области квартиры с большей площадью, большим количеством комнат и меньшим количеством балконов продаются дольше. На объявления со сроком активности 1-3 мес. приходится 33,2% от общего числа всех объявлений по Ленинградской области – это самая большая группа, далее следует группа со сроком от 6 мес. на которую приходится 32,2%.

4. Цена за кв. метр и общая площадь квартир в Санкт-Петербурге выше, чем в Ленинградской области по всем сегментам активности объявлений. Квартиры с меньшей площадью как в Санкт-Петербурге так и в Ленинградской области продаются быстрее, однако если в Санкт-Петербурге цена за кв. метр растет с увеличением срока активности объявления, то в Ленинградской области есть аномалия в виде самой высокой цены за кв. метр в первой категории сегмента (активность до 1 мес.). Возможно, это ближайший пригород и цена растет за счет того, что покупатели выбирают область вместо спальных районов Санкт-Петербурга. В Ленинградской области нет четкой связи сегмента активности объявления от количества парков и водоемов, в отличии от Санкт-Петербурга - это может быть связано с тем, что область, итак, находится в более «зеленой зоне».

---
### Задача 2. Сезонность объявлений

Заказчику важно понять сезонные тенденции на рынке недвижимости Санкт-Петербурга и Ленинградской области — то есть для всего региона, чтобы выявить периоды с повышенной активностью продавцов и покупателей недвижимости. Это поможет спланировать маркетинговые кампании и выбрать сроки для выхода на рынок.

Результат запроса должен ответить на такие вопросы:
1. В какие месяцы наблюдается наибольшая активность в публикации объявлений о продаже недвижимости? А в какие — по снятию?
2. Совпадают ли периоды активной публикации объявлений и периоды, когда происходит повышенная продажа недвижимости (по месяцам снятия объявлений)?
3. Как сезонные колебания влияют на среднюю стоимость квадратного метра и среднюю площадь квартир? 

```postgresql
-- Определим аномальные значения (выбросы) по значению перцентилей.
WITH limits AS 
    (
    SELECT  
        PERCENTILE_DISC(0.99) WITHIN GROUP (ORDER BY total_area) AS total_area_limit,
        PERCENTILE_DISC(0.99) WITHIN GROUP (ORDER BY rooms) AS rooms_limit,
        PERCENTILE_DISC(0.99) WITHIN GROUP (ORDER BY balcony) AS balcony_limit,
        PERCENTILE_DISC(0.99) WITHIN GROUP (ORDER BY ceiling_height) AS ceiling_height_limit_h,
        PERCENTILE_DISC(0.01) WITHIN GROUP (ORDER BY ceiling_height) AS ceiling_height_limit_l
    FROM real_estate.flats     
    ),
-- Найдём id объявлений, которые не содержат выбросы.
filtered_id AS 
    (
    SELECT id
    FROM real_estate.flats  
    WHERE 
        total_area < (SELECT total_area_limit FROM limits)
        AND (rooms < (SELECT rooms_limit FROM limits) OR rooms IS NULL)
        AND (balcony < (SELECT balcony_limit FROM limits) OR balcony IS NULL)
        AND ((ceiling_height < (SELECT ceiling_height_limit_h FROM limits)
        AND ceiling_height > (SELECT ceiling_height_limit_l FROM limits)) OR ceiling_height IS NULL)
    ),
-- В данные добавили дату продажи, общую площадь и стоимость.
tab_full AS 
    (
    SELECT 
    	f.id,
    	f.total_area,
    	a.last_price,
    	a.first_day_exposition,
     	CASE 
     	 	WHEN a.days_exposition IS NOT NULL
     			THEN a.first_day_exposition + a.days_exposition::INT 
     		ELSE NULL
     	END AS date_sale
    FROM real_estate.flats AS f
    JOIN real_estate.advertisement AS a USING(id) 
-- Убрали объявления с выбросами и оставили города
    WHERE 
        f.id IN (SELECT * FROM filtered_id) 
    	AND f.type_id IN (SELECT type_id FROM real_estate.type WHERE type='город')
    ),
-- Посчитали количество снятых объявлений по каждому месяцу
-- и проранжировали мес. по кол-ву снятых объявлений (1 - больше всего объявлений снято).
-- Добавили среднюю общую площадь и среднюю цену за кв.м.
month_sale AS 
    (
    SELECT
    	EXTRACT(MONTH FROM date_sale) AS number_month,
    	RANK() OVER (ORDER BY COUNT(id) DESC) AS rank_sold_adv,
    	COUNT(id) AS count_sold_adv,
    	ROUND(COUNT(id) / SUM(COUNT(id)) OVER() * 100.0, 1) AS persent_sold_adv,
    	ROUND(AVG(last_price / total_area::NUMERIC)) AS avg_cost_m_sold,
    	ROUND(AVG(total_area::NUMERIC), 1) AS avg_total_area_sold
    FROM tab_full
-- Оставили только снятые объявления и за полные годы 2015-2018
    WHERE 
        date_sale IS NOT NULL 
        AND date_sale >= '2015-01-01' 
        AND date_sale <= '2018-12-31'
    GROUP BY EXTRACT(MONTH FROM date_sale)
    ),
-- Посчитали количество опубликованных объявлений по каждому месяцу 
-- и проранжировали мес. по кол-ву объявлений (1 - больше всего объявлений).
-- Добавили среднюю общую площадь и среднюю цену за кв.м.
month_publ AS 
    ( 
    SELECT
    	EXTRACT(MONTH FROM first_day_exposition) AS number_month,
    	TO_CHAR(first_day_exposition, 'TMmon') AS months,
    	RANK() OVER (ORDER BY COUNT(id) DESC) AS rank_publ_adv,
    	COUNT(id) AS count_publ_adv,
    	ROUND(COUNT (id) / SUM(COUNT(id)) OVER() * 100.0, 1) AS persent_publ_adv,
    	ROUND(AVG(last_price / total_area::NUMERIC)) AS avg_cost_m_publ_adv,
    	ROUND(AVG(total_area::NUMERIC), 1) AS avg_total_area_publ_adv
    FROM tab_full AS tf
-- Оставили опубликованные объявления за полные годы 2015-2018
    WHERE 
        first_day_exposition >= '2015-01-01' 
        AND first_day_exposition <= '2018-12-31'
    GROUP BY EXTRACT(MONTH FROM first_day_exposition), TO_CHAR(first_day_exposition, 'TMmon')
    )
-- Выводим общие данные
SELECT
	months,
	rank_publ_adv,
	count_publ_adv,
	persent_publ_adv,
	avg_cost_m_publ_adv,
	avg_total_area_publ_adv,
	rank_sold_adv,
	count_sold_adv,
	persent_sold_adv,
	avg_cost_m_sold,
	avg_total_area_sold
FROM month_publ
JOIN month_sale USING(number_month)
ORDER BY number_month;

```

**Результат:**

months|rank_publ_adv|count_publ_adv|persent_publ_adv|avg_cost_m_publ_adv|avg_total_area_publ_adv|rank_sold_adv|count_sold_adv|persent_sold_adv|avg_cost_m_sold|avg_total_area_sold|
------|-------------|--------------|----------------|-------------------|-----------------------|-------------|--------------|----------------|---------------|-------------------|
jan   |           12|           735|             5.2|           106106.0|                   59.2|            7|           878|             7.3|       103841.0|               57.6|
feb   |            3|          1369|             9.7|           103059.0|                   60.1|           11|           741|             6.1|       100879.0|               59.6|
mar   |            8|          1119|             8.0|           102430.0|                   60.0|            8|           823|             6.8|       105377.0|               58.6|
apr   |           10|          1021|             7.3|           102632.0|                   60.6|           10|           769|             6.4|       100222.0|               56.6|
may   |           11|           891|             6.3|           102465.0|                   59.2|           12|           716|             5.9|        99555.0|               57.8|
jun   |            5|          1224|             8.7|           104802.0|                   58.4|            9|           782|             6.5|       101913.0|               60.2|
jul   |            7|          1149|             8.2|           104489.0|                   60.4|            6|          1130|             9.4|       102505.0|               59.0|
aug   |            6|          1166|             8.3|           107035.0|                   59.0|            5|          1145|             9.5|       100057.0|               57.0|
sep   |            4|          1341|             9.5|           107563.0|                   61.0|            3|          1247|            10.3|       104397.0|               57.8|
oct   |            2|          1437|            10.2|           104065.0|                   59.4|            1|          1367|            11.3|       104609.0|               58.9|
nov   |            1|          1569|            11.2|           105049.0|                   59.6|            2|          1307|            10.8|       103882.0|               56.9|
dec   |            9|          1024|             7.3|           104775.0|                   58.8|            4|          1179|             9.8|       105713.0|               59.4|



1. В ноябре, октябре и феврале наблюдается наивысшая активность по публикации объявлений. Январь, май и апрель – это месяцы с наименьшей активностью по публикации объявлений.
Больше всего объявлений было снято: в октябре, ноябре и сентябре, а меньше всего: в мае, феврале и апреле.
2. Октябрь и ноябрь совпали по активности публикаций и снятию объявлений. Май можно отнести к наименее активному месяцу по подачи и снятию объявлений.
3. В течение года средние величины стоимости за квадратный метр и площади по снятым объявлениям, оказались незначительно ниже аналогичных величин по опубликованным объявлениям.
В сентябре можно наблюдать наивысшую среднюю цену за квадратный метр и максимальную среднюю площадь среди опубликованных объявлений – это один из месяцев с наивысшей активностью по снятию объявлений. С марта по май наблюдается период с низкой средней ценой за кв. метр, этот же период совпадает с невысокой активностью по снятию объявлений. 
Наивысшая средняя цена за квадратный метр среди снятых объявлений была зафиксирована в марте, этот месяц не выделяется по активности среди опубликованных или снятых объявлений. В мае наблюдается наименьшая средняя цена за квадратный метр среди снятых объявлений, это месяц наименьшей активности по снятым объявлениям и низкой активности по опубликованным объявлениям.
В целом изменение средней цены за кв. метр в течение года составило не более 3,1% относительно средней величины, а изменение средней площади не более 3,3% относительно средней величины как по опубликованным, так и по снятым объявлениям.

---
### Задача 3. Анализ рынка недвижимости Ленобласти

Заказчик хочет определить, в каких населённых пунктах Ленинградской области активнее всего продаётся недвижимость и какая именно. Так он увидит, где стоит поработать, и учтёт особенности Ленинградской области при принятии бизнес-решений.

Результат запроса должен ответить на такие вопросы:
1. В каких населённые пунктах Ленинградской области наиболее активно публикуют объявления о продаже недвижимости?
2. В каких населённых пунктах Ленинградской области — самая высокая доля снятых с публикации объявлений? 
3. Какова средняя стоимость одного квадратного метра и средняя площадь продаваемых квартир в различных населённых пунктах? 
4. Среди выделенных населённых пунктов какие пункты выделяются по продолжительности публикации объявлений? То есть где недвижимость продаётся быстрее, а где — медленнее.

```postgresql
-- Определим аномальные значения (выбросы) по значению перцентилей:
WITH limits AS 
    (
    SELECT  
        PERCENTILE_DISC(0.99) WITHIN GROUP (ORDER BY total_area) AS total_area_limit,
        PERCENTILE_DISC(0.99) WITHIN GROUP (ORDER BY rooms) AS rooms_limit,
        PERCENTILE_DISC(0.99) WITHIN GROUP (ORDER BY balcony) AS balcony_limit,
        PERCENTILE_DISC(0.99) WITHIN GROUP (ORDER BY ceiling_height) AS ceiling_height_limit_h,
        PERCENTILE_DISC(0.01) WITHIN GROUP (ORDER BY ceiling_height) AS ceiling_height_limit_l
    FROM real_estate.flats     
    ),
-- Найдём id объявлений, которые не содержат выбросы:
filtered_id AS 
    (
    SELECT id
    FROM real_estate.flats  
    WHERE 
        total_area < (SELECT total_area_limit FROM limits)
        AND (rooms < (SELECT rooms_limit FROM limits) OR rooms IS NULL)
        AND (balcony < (SELECT balcony_limit FROM limits) OR balcony IS NULL)
        AND ((ceiling_height < (SELECT ceiling_height_limit_h FROM limits)
        AND ceiling_height > (SELECT ceiling_height_limit_l FROM limits)) OR ceiling_height IS NULL)
    )
-- Убрали объявления с выбросами и Санкт-Петербург, оставили нас.пункты с количеством объявлений более 100.
-- Расчитали основные показатели. 
SELECT
	c.city,
	COUNT(id) AS count_adv,
	ROUND(COUNT(id) FILTER (WHERE a.days_exposition IS NOT NULL) / COUNT(id)::NUMERIC, 3) share_sales,
	ROUND(AVG(a.last_price / f.total_area::NUMERIC) FILTER (WHERE a.days_exposition IS NOT NULL)) AS avg_cost_meter,
	ROUND(AVG(f.total_area::NUMERIC) FILTER (WHERE a.days_exposition IS NOT NULL)) AS avg_total_area,
	ROUND(AVG(a.days_exposition::NUMERIC)) AS avg_days_to_sell
FROM real_estate.flats AS f
JOIN real_estate.advertisement AS a USING(id)
JOIN real_estate.city AS c USING(city_id)
WHERE 
    f.id IN (SELECT * FROM filtered_id) 
    AND c.city <> 'Санкт-Петербург'
GROUP BY c.city
HAVING COUNT(id) > 100
ORDER BY count_adv DESC;

```

**Результат:**

city           |count_adv|share_sales|avg_cost_meter|avg_total_area|avg_days_to_sell|
---------------|---------|-----------|--------------|--------------|----------------|
Мурино         |      568|      0.937|       85655.0|            44|             149|
Кудрово        |      463|      0.937|       94583.0|            46|             161|
Шушары         |      404|      0.926|       78110.0|            54|             152|
Всеволожск     |      356|      0.857|       68796.0|            54|             190|
Парголово      |      311|      0.926|       89711.0|            51|             156|
Пушкин         |      278|      0.831|      103239.0|            58|             197|
Гатчина        |      228|      0.890|       67978.0|            50|             188|
Колпино        |      227|      0.921|       74645.0|            52|             147|
Выборг         |      192|      0.875|       57684.0|            54|             182|
Петергоф       |      154|      0.883|       83992.0|            50|             197|
Сестрорецк     |      149|      0.899|      103957.0|            61|             215|
Красное Село   |      136|      0.897|       71672.0|            52|             206|
Новое Девяткино|      120|      0.883|       76279.0|            51|             176|
Сертолово      |      117|      0.863|       68509.0|            54|             174|
Бугры          |      104|      0.875|       80118.0|            47|             156|

1. Для исследования оставили населенные пункты, в которых количество публикуемых объявлений составило более 100. Данные города охватывают 60% всех объявлений Ленинградской области. В результате получили список из 15 населенных пунктов. Далее проанализируем их более детально.
2. Во всех населенных пунктах процент снятых объявлений выше 83%, а в первой пятерке выше 92%. Также стоит отметить, что Кудрово и Мурино попали в топ 2 как по количеству опубликованных, так и доли снятых объявлений.
3. Общая средняя стоимость кв. метра по всем анализируемым населенным пунктам  составила 78'110 руб. Наиболее высока стоимость оказалась в Сестрорецке и Пушкине, наименьшая в Выборге и Гатчине. Максимальная средняя стоимость за один кв.метр превысила минимальную на 46'273 руб. или на 80%. 
Общая средняя площадь по всем анализируемым населенным пунктам составила 52 кв.м. Максимальная средняя площадь превысила минимальную на 17 кв.м. или на 39%.
Пушкин и Сестрорецк оказались лидерами по стоимости кв. метра и средней площади квартир среди представленного списка населенных пунктов.
4. Общая средняя продолжительность по всем топ-15 составила 176 дней. Самая короткая продолжительность в Колпино и Мурино, самая большая в Сестрорецке и Красном Селе. Мурино оказалось лучшим населенным пунктом по скорости и доли снятия объявлений.

---
### Общие выводы

В Санкт-Петербурге в течение месяца продаются менее 20% квартир, в Ленинградской области этот показатель еще ниже - менее 15% квартир. В обоих исследуемых субъектах почти половина всех продаж осуществляется в срок до трех месяцев. В Санкт-Петербурге и области быстрее продаются квартиры с небольшой площадью. В Санкт-Петербурге на скорость продаж влияет количество парков и водоемом поблизости, чем их больше, тем быстрее продаются квартиры, в Ленинградской области такой зависимости не наблюдается. Также на скорость продаж в обоих субъектах влияет количество балконов, чем их больше, тем быстрее продаются квартиры. Количество комнат, напротив, увеличивает время продажи в обоих регионах, чем их больше, тем дольше продается квартира. 

В Санкт-Петербурге и Ленинградской области наиболее активный период по продажам с сентября по ноябрь, наименее – январь и с апреля по май.

В Ленинградской области стоит обратить внимание на Мурино, Кудрово и Шушары здесь на ряду с наибольшим количеством объявлений, высокая доля и скорость продаж. Также стоит учесть, что в Пушкине и Сестрорецке квартиры с наибольшей средней площадью и самым высоким уровнем стоимости кв. метра, эти населенные пункты характеризуются низкой скоростью продажи.
